# DiploDatos 2020 - Coronavirus en Argentina

## Introducción al Machine Learning - Clasificación y regresión logística


In [102]:
### Aumentar el ancho del notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [103]:
# Importamos librerías necesarias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [104]:
from sklearn.model_selection import train_test_split

### Regresión Lineal

Probaremos ajustar los puntos usando una recta.

In [105]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [106]:
# Create linear regression object
regr = linear_model.LinearRegression()

### Clasificación
Para la clasificación, se trabajará sobre el dataset de personas fallecidas por covid-19.

In [107]:
url = 'https://raw.githubusercontent.com/lucia15/Datos-Covid19-Argentina/master/Argentina-covid19-fallecidos.csv'

data = pd.read_csv(url, error_bad_lines=False)

Se utilizarán las columnas "provincia", "género" y "edad". Para ello se quitarán previamente las columnas restantes.

In [108]:
data = data.drop(columns=['fecha', 'num_caso', 'tipo_caso', 'comorbilidades', 'viajes', 'observaciones'])

data = data.dropna()

data

,provincia,genero,edad
0,CABA,hombre,64.0
1,Chaco,hombre,61.0
2,CABA,hombre,64.0
3,Buenos Aires,mujer,67.0
5,Chaco,hombre,53.0
...,...,...,...
2277,Río Negro,hombre,87.0
2278,Buenos Aires,mujer,74.0
2279,Buenos Aires,mujer,61.0
2280,Buenos Aires,mujer,64.0


Luego, se divide en 4 rangos la variable edad: de 0 a 30 años, de 31 a 60, de 61 a 80 y de 81 a 100. Con estos 4 rangos definidos, se agregará una nueva columna que defina a qué rango pertenece cada registro.

In [109]:
criteria = [data['edad'].between(0, 30), data['edad'].between(31, 60), data['edad'].between(61, 80), data['edad'].between(81, 100)]
values = ['0-30', '31-60', '61-80', '81-100']


In [110]:
data['rango-edad'] = np.select(criteria, values, 0)

In [111]:
data = data.drop(['edad'], axis=1)

### Tratamiento de género y rango-edad
Como se trata de **variables categóricas**, en la cuál no hay una relación de orden entre las categorías, necesitamos aplicar un **One Hot Encoding**.

Se transformarán las variables "género" y "rango-edad."

In [112]:
data = pd.concat([data, pd.get_dummies(data['genero'], prefix='genero')],axis=1).drop(['genero'], axis=1)

In [113]:
data = pd.concat([data, pd.get_dummies(data['rango-edad'], prefix='rango-edad')],axis=1).drop(['rango-edad'], axis=1)

In [114]:
data = data.drop(['rango-edad_0'], axis=1)
data.head()

,provincia,genero_hombre,genero_mujer,genero_sin dato,rango-edad_0-30,rango-edad_31-60,rango-edad_61-80,rango-edad_81-100
0,CABA,1,0,0,0,0,1,0
1,Chaco,1,0,0,0,0,1,0
2,CABA,1,0,0,0,0,1,0
3,Buenos Aires,0,1,0,0,0,1,0
5,Chaco,1,0,0,0,1,0,0


### Selección de features y variable objetivo

**variable objetivo: 'provincia'**
Se buscará predecir la provincia en base al género y los rangos de edad.

In [115]:
feature_names = ['genero_hombre', 'genero_mujer', 'rango-edad_0-30', 'rango-edad_31-60', 'rango-edad_61-80', 'rango-edad_81-100']

X = data[feature_names]
y = data['provincia']

In [116]:
X.shape, y.shape

((2264, 6), (2264,))

### División en etrenamiento y evaluación

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [118]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1698, 6), (566, 6), (1698,), (566,))

### Logistic Regression

In [119]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

print('Accuracy of Logistic regression classifier on training set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of Logistic regression classifier on training set: 0.49
Accuracy of Logistic regression classifier on test set: 0.51


In [120]:
from sklearn.metrics import classification_report

y_train_pred = logreg.predict(X_train)
y_test_pred = logreg.predict(X_test)

print(classification_report(y_test, y_test_pred))

                  precision    recall  f1-score   support

    Buenos Aires       0.51      1.00      0.67       288
            CABA       0.00      0.00      0.00       208
           Chaco       0.00      0.00      0.00        29
         Córdoba       0.00      0.00      0.00        10
        La Rioja       0.00      0.00      0.00         2
         Mendoza       0.00      0.00      0.00         4
        Misiones       0.00      0.00      0.00         1
         Neuquén       0.00      0.00      0.00         6
       Río Negro       0.00      0.00      0.00        12
           Salta       0.00      0.00      0.00         1
        Santa Fe       0.00      0.00      0.00         1
Tierra del Fuego       0.00      0.00      0.00         1
         Tucumán       0.00      0.00      0.00         3

        accuracy                           0.51       566
       macro avg       0.04      0.08      0.05       566
    weighted avg       0.26      0.51      0.34       566

